In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora
from gensim import models
import numpy as np
import ast

In [2]:
df_snippets = pd.read_csv('data/02_bow_snippets_claims.csv')

In [3]:
## We evaluate strings as list of words

df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda st: ast.literal_eval(st))

In [4]:
## For each dictionary
## - We transform list of words from snippets into bow indices
## - We construct a tf-idf model

for nb_word in sorted(list(np.arange(10000,40000,5000)),reverse=True):
    dictionary = corpora.Dictionary.load('dictionaries/02_'+str(nb_word/1000)+'.dict')
    df_snippets['snippet_bow_'+str(nb_word/1000)] = df_snippets['snippet_content'].apply(lambda x: dictionary.doc2bow(x))
    tfidf = models.TfidfModel(list(df_snippets['snippet_bow_'+str(nb_word/1000)].as_matrix()))
    df_snippets['snippet_tfidf_'+str(nb_word/1000)] = df_snippets['snippet_bow_'+str(nb_word/1000)].apply(lambda x: tfidf[x])
    tfidf.save('topic_models/03_'+str(nb_word/1000)+'.tfidf')

In [5]:
## For each dictionary
##    For each chosen topic model number (from 100 to 300 with a step of 50)
##        We build lsi, rp and lda models

for nb_word in sorted(list(np.arange(10000,40000,5000)),reverse=True):
    dictionary = corpora.Dictionary.load('dictionaries/factchecking_'+str(nb_word/1000)+'.dict')
    for nb_topic in np.arange(100,350,50):
        lsi = models.LsiModel(list(df_snippets['snippet_tfidf_'+str(nb_word/1000)].as_matrix()), 
                              id2word=dictionary, num_topics=nb_topic)
        lsi.save('topic_models/03_'+str(nb_word/1000)+'_'+str(nb_topic)+'.lsi')
        rp = models.RpModel(list(df_snippets['snippet_bow_'+str(nb_word/1000)].as_matrix()), 
                            num_topics=nb_topic)
        rp.save('topic_models/03_'+str(nb_word/1000)+'_'+str(nb_topic)+'.rp')
        lda = models.LdaModel(list(df_snippets['snippet_bow_'+str(nb_word/1000)].as_matrix()), 
                              id2word=dictionary, num_topics=nb_topic)
        lda.save('topic_models/03_'+str(nb_word/1000)+'_'+str(nb_topic)+'.lda')

/home/ymerej/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)
